In [315]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

In [296]:
data = pd.read_csv('../../../data_analysis/heatAlerts.csv')

In [297]:
df = data[data['outcome'] != "."]

In [298]:
for i in range(1,10):
    df = df.drop(['Unnamed: 4' + str(i)], axis = 1)

In [299]:
def trim_all_columns(df):
    """
    Trim whitespace from ends of each value across all series in dataframe
    """
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(trim_strings)
df = trim_all_columns(df)

In [300]:
df = df[df['SB_HTENDS'] != '.']

In [301]:
df.dropna(subset = ['ID','DIMatheat', 'SB_PROB', 'Heat Probability Category', 'DC_ACLEV', 'DC_ACTIM'], inplace = True)

In [302]:
df2 = pd.DataFrame(columns = df.columns)
for index, row in df.iterrows():
    try:
        i = int(row['DC_ACLEV'])
#         print(row)
        df2.loc[len(df2.index)] = row
    except: 
        continue

In [303]:
df = df2

# ML

In [304]:
data_with_outcome = df
df['outcome'].head()

0    O
1    P
2    O
3    O
4    P
Name: outcome, dtype: object

In [305]:
data_with_outcome = data_with_outcome[(data_with_outcome['outcome']=="P") | (data_with_outcome['outcome']=="O") | (data_with_outcome['outcome']=="A")]
new_sb_htstarts = pd.to_timedelta(data_with_outcome['SB_HTSTARTS'])
new_sb_htends = pd.to_timedelta(data_with_outcome['SB_HTENDS'])  
data_with_outcome['time_elapsed'] = new_sb_htends - new_sb_htstarts
data_with_outcome['sc_elapsed'] = data_with_outcome.time_elapsed.dt.total_seconds()
data_with_outcome.dropna(subset = ['sc_elapsed'], inplace = True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

In [306]:
Y = data_with_outcome['outcome']
Y.head()

0    O
1    P
2    O
3    O
4    P
Name: outcome, dtype: object

In [307]:
X = data_with_outcome.drop(columns=['outcome'])
X['DC_ACLEV'] = X['DC_ACLEV'].astype(int)
X['DC_ACTIM'] = X['DC_ACTIM'].astype(int)

In [308]:
Y[Y=="P"] = 1
Y[Y=="A"] = 1  # aborted is also pregnant
Y[Y=="O"] = 0
Y = Y.astype(int)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/series.py:992: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https:

In [309]:
X = X[['DIMatheat', 'SB_PROB', 'Heat Probability Category', 'DC_ACLEV', 'DC_ACTIM', 'sc_elapsed']]

In [310]:
x_train, x_test, y_train, y_test = train_test_split(X.to_numpy(), Y.to_numpy(), test_size=0.33, random_state=42)

In [311]:
x_train

array([[ 1.150e+02,  1.000e+02,  5.000e+00,  1.000e+02,  1.700e+01,
        -3.240e+04],
       [ 5.400e+01,  9.800e+01,  4.000e+00,  9.800e+01,  1.300e+01,
        -3.606e+04],
       [ 1.560e+02,  9.600e+01,  4.000e+00,  1.000e+02,  2.000e+00,
         5.766e+04],
       ...,
       [ 1.040e+02,  9.200e+01,  4.000e+00,  9.200e+01,  2.000e+01,
        -4.320e+04],
       [ 7.600e+01,  7.800e+01,  3.000e+00,  7.600e+01,  2.200e+01,
        -4.320e+04],
       [ 7.900e+01,  1.000e+02,  5.000e+00,  1.000e+02,  1.600e+01,
        -2.514e+04]])

In [312]:
clf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=20, random_state=0)
clf.fit(x_train, y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2),
                   n_estimators=20, random_state=0)

In [313]:
pred_train = clf.predict(x_train)
pred_test = clf.predict(x_test)
print(f"Training accuracy: {accuracy_score(y_train, pred_train)}")
print(f"Testing accuracy: {accuracy_score(y_test, pred_test)}")

Training accuracy: 0.8424908424908425
Testing accuracy: 0.5259259259259259


# neural network

In [316]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

In [317]:
clf.fit(X, Y)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 2), random_state=1,
              solver='lbfgs')

In [318]:
pred_train = clf.predict(x_train)
pred_test = clf.predict(x_test)
print(f"Training accuracy: {accuracy_score(y_train, pred_train)}")
print(f"Testing accuracy: {accuracy_score(y_test, pred_test)}")

Training accuracy: 0.5714285714285714
Testing accuracy: 0.5703703703703704
